# einfaldur filter

In [1]:
import re
def simple_cleaner (sent):
    sent = re.sub(r"[^ \w.]", "",sent)
    if sent == "": return
    sent = re.sub(r"[\n\t\r]*", "", sent)
    sent = re.sub(r"\s+", " ", sent)
    return sent

# Snerpa.is

In [2]:
from bs4 import BeautifulSoup
import urllib3
import json

def snerpa():
    with open ('./scraping/snerpa_links.json') as f:
        snerpa_links = json.load(f)

    page_data = {}
    num = len(snerpa_links)

    for (i, link) in enumerate(snerpa_links, 1):
        page = urllib3.request("GET", link)
        html_data = page.data
        soup = BeautifulSoup(html_data, 'html.parser')

        all_text = soup.find('article')
        author = all_text.find('em')
        if author != None:
            author.extract()
        
        for br in soup.find_all("br"):
            br.replace_with("\n")

        text = [t for t in re.split(r"(\n)", all_text.text) if t != ""]
        text = [t for t in list(map(simple_cleaner, text)) if t != None]

        as_string = "\n".join(text[1:])
        page_data[text[0]] = as_string
        print(f"{i}/{num}", end='\r')

    return page_data

# Sol.heimsnet.is

In [2]:
from bs4 import BeautifulSoup
import urllib3
import re

def heimsnet():
    page_data = {}
    
    with open('./solnet.txt') as f:
        log = f.read().split('\n')

    counter = 0
    for line in log:
        if line == "":
            counter += 1
            continue
        link = line.split(' ')[-1]
        page = urllib3.request("GET", link)
        html_data = page.data
        soup = BeautifulSoup(html_data, 'html.parser')

        all_text = soup.find('article')
        author = all_text.find('em')
        if author != None:
            author.extract()
        
        for br in soup.find_all("br"):
            br.replace_with("\n")

        text = [t for t in re.split(r"(\n)", all_text.text) if t != ""]
        text = [t for t in list(map(simple_cleaner, text)) if t != None]

        as_string = "\n".join(text[1:])
        page_data[text[0]] = as_string

heimsnet()

['Af litlum neista',
 '',
 'Oft má finna atvik smátt',
 'sem orsök nýrra kynna.',
 'Það markar oft þá meginátt',
 'sem mannsins örlög þráðinn spinna.',
 'Sígarettu hafði hann',
 'í hendi milli fingra sinna,',
 'en eldspýturnar eigi fann,',
 'þær einhversstaðar varð að finna.',
 '',
 'Ástin er eins og sinueldur.',
 'Ástin er segulstál.',
 'Af litlum neista verður oft mikið bál.',
 'Ástin er eins og hvítigaldur,',
 'gagntekur líkama’ og sál.',
 'Af litlum neista verður oft mikið bál.',
 '',
 'Brátt hann leit á bjarta mey',
 'við borð í salnum aðeins innar.',
 'Hann óðamála sagði ókey!',
 'Hún er yndisfögur með rjóðar kinnar.',
 'Hann slátt í brjósti sínu kenndi',
 'er sá hann þessa fögru snót,',
 'og beina leið á borðið renndi',
 'með bros á vör og hljóp við fót.',
 'Ástin er eins og sinueldur...',
 '',
 'En ekki bara eld á spýtu',
 'einan fékk hjá snótinni.',
 'Því sjá þar mátti svart á hvítu',
 'hvað skein frá hjartarótinni.',
 'Nú boga amors englar spenntu,',
 'örvahríðin óx nú fljótt

In [4]:
import json
page_data = {**snerpa(), **heimsnet()}
with open('./scraped_data.json', 'w', encoding="utf-8") as f:
    json.dump(page_data, f, ensure_ascii=False, indent=2)

In [80]:
import json
page_data = {**heimsnet()}
with open('./scraped_data.json', 'w', encoding="utf-8") as f:
    json.dump(page_data, f, ensure_ascii=False, indent=2)

['Af litlum neista', 'Söngurinn hennar Siggu', 'Afmælisdigtur', 'Allir hanar gala', 'Allir þurfa að eiga vin', 'Allt fram streymir', 'Amma kveður', 'Anna í Hlíð', 'Álfareiðin', 'Ánamaðkadansinn', 'Bangsímon', 'Barn', 'Bátasmiðurinn', 'Bjallan hringir búin stundin', 'Bjössi á mjólkurbílnum', ' Sigling', 'Blómálfadansinn', 'Dagný', 'Dagur er liðinn', None, 'Dansa ei við ungfrú Línu', None, None, 'Dansi dansi dúkkan mín', None, None, 'Dátt er blessað lognið', 'Do Re Mí', 'Dvel ég í draumahöll', 'Dvergar úr björgum.', 'Ef væri ég söngvari', 'Ef þú giftist', 'Ef þú kætist', ' ', 'Ein fögur eik', 'Einu sinni á ágústkvöldi', 'Eldgamla Ísafold', 'Elskulega mamma mín', None, 'Endurvinnslan', 'En það sólskin', 'Epli og perur', 'Er faðir barna og blóma', 'Erla góða Erla', 'Ég á lítinn skrýtinn skugga', 'Ég bið að heilsa', 'Ég er fæddur ferðamaður', None, 'Furðuverk', None, 'Ég á gamla frænku', 'Ég á lítinn kátan kött', 'Ég get klappað', None, 'Ég heiti eftir afa mínum', None, None, 'Ég heyri svo 

In [85]:
page = urllib3.request("GET", "https://sol.heimsnet.is/!GSS_ymislegt/Lagatextar2.htm")
html_data = page.data
soup = BeautifulSoup(html_data, 'html.parser')
raw_text = soup.find('td', {"valign": "top", "style": "margin:0; padding:0; font-family: Book Antiqua; color: #000000; word-spacing: 0; line-height: 200%; text-indent:0"})


In [90]:
text = [simple_cleaner(t) for t in raw_text.text.split('\n') if t != ""]
text = [t for t in text if t != None]
current_title = ""
current_lyrics = []

with open('./solnet.json', 'w', encoding="utf-8") as f:
    json.dump(text, f, ensure_ascii=False, indent=2)
